In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-data/"))

['test_df_h.pkl', 'test_df_c.pkl', 'train_df_w.pkl', 'test_df_w.pkl', 'train_df_s.pkl', 'test_df_s.pkl', 'test_df_r.pkl', 'train_df_h.pkl', 'test_identity.pkl', 'train_df_r.pkl', 'train_identity.pkl', 'train_df_c.pkl']


In [2]:
train = pd.read_pickle('../input/mini-model-features/train_features_c.pkl')
test= pd.read_pickle('../input/mini-model-features/test_features_c.pkl')

train_y = train['isFraud'].copy()

print(train.shape)
print(train_y.shape)
print(test.shape)

(68519, 412)
(68519,)
(69266, 412)


In [3]:
train[features_columns].head()

,isFraud,TransactionDT,TransactionAmt,addr1,addr2,dist2,D1,D2,D3,D4,D5,D6,D7,D8,D14,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V50,V51,V52,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V66,V67,V71,V72,V73,V74,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V92,V93,V95,V96,V97,V101,V102,V103,V123,V124,V125,V126,V127,V128,V132,V133,V134,V135,V136,V137,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V284,V285,V287,V291,V294,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_21,id_31,id_38,DeviceInfo,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,addr1_isnull,addr2_isnull,dist2_isnull,D2_isnull,D3_isnull,D5_isnull,D7_isnull,D8_isnull,D9_isnull,D14_isnull,V217_isnull,V218_isnull,V219_isnull,V223_isnull,V224_isnull,V225_isnull,V228_isnull,V229_isnull,V230_isnull,V231_isnull,V232_isnull,V233_isnull,V235_isnull,V236_isnull,V237_isnull,V240_isnull,V241_isnull,V242_isnull,V243_isnull,V244_isnull,V246_isnull,V247_isnull,V248_isnull,V249_isnull,V252_isnull,V253_isnull,V254_isnull,V257_isnull,V258_isnull,V260_isnull,V261_isnull,V262_isnull,V263_isnull,V264_isnull,V265_isnull,V266_isnull,V267_isnull,V268_isnull,V269_isnull,V273_isnull,V274_isnull,V275_isnull,V276_isnull,V277_isnull,V278_isnull,id_03_isnull,id_04_isnull,id_07_isnull,id_08_isnull,id_09_isnull,id_10_isnull,id_14_isnull,id_21_isnull,id_22_isnull,id_24_isnull,id_25_isnull,id_26_isnull,C1_target_code,C1_fq_enc,C2_target_code,C2_fq_enc,C4_target_code,C4_fq_enc,C6_target_code,C6_fq_enc,C7_target_code,C7_fq_enc,C8_target_code,C8_fq_enc,C10_target_code,C10_fq_enc,C11_target_code,C11_fq_enc,C12_target_code,C12_fq_enc,C13_target_code,C13_fq_enc,C14_target_code,C14_fq_enc,card1_target_code,card1_fq_enc,card2_target_code,card2_fq_enc,card3_target_code,card3_fq_enc,card4_target_code,card4_fq_enc,card5_target_code,card5_fq_enc,card6_target_code,card6_fq_enc,D9_target_code,D9_fq_enc,device_name_target_code,device_name_fq_enc,DeviceType_target_code,DeviceType_fq_enc,id_13_target_code,id_13_fq_enc,id_14_target_code,id_14_fq_enc,id_15_target_code,id_15_fq_enc,id_16_target_code,id_16_fq_enc,id_17_target_code,id_17_fq_enc,id_18_target_code,id_19_target_code,id_20_target_code,id_22_target_code,id_22_fq_enc,id_23_target_code,id_23_fq_enc,id_24_target_code,id_24_fq_enc,id_25_fq_enc,id_26_target_code,id_26_fq_enc,id_27_target_code,id_27_fq_enc,id_28_target_code,id_28_fq_enc,id_29_target_code,id_29_fq_enc,lastest_browser_target_code,lastest_browser_fq_enc,id_36_target_code,id_36_fq_enc,id_37_target_code,id_37_fq_enc,P_emaildomain_target_code,P_emaildomain_fq_enc,R_emaildomain_target_code,R_emaildomain_fq_enc,cents_target_code,cents_fq_enc,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,D1_day_mean,D1_week_mean,D2_day_mean,D2_week_mean,D3_day_mean,D3_week_mean,D4_day_mean,D4_week_mean,D5_day_mean,D5_week_mean,D6_day_mean,D6_week_mean,D7_day_mean,D7_week_mean,D8_day_mean,D8_week_mean,D10_day_mean,D10_week_mean,D12_day_mean,D12_week_mean,D13_day_mean,D13_week_mean,D14_day_mean,D14_week_mean,D15_day_mean,D15_week_mean
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [4]:
rm_cols = [
    'isFraud', 'TransactionDT', 'DT', 'addr2', 'C3', 'TransactionAmt',
    'addr1', 'addr2'
    'id_21', 'id_30', 'id_31', 'id_34','id_38'
    'V27', 'V28', 'V65', 'V68','V88', 'V89',
    'DeviceInfo', 
]
features_columns = list(train)
for col in rm_cols:
    if col in features_columns:
        features_columns.remove(col)

In [5]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':1000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                    'early_stopping_rounds':100, 
                } 

In [6]:
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)

Fold: 0  -  51744 16775
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[639]	training's auc: 1	valid_1's auc: 0.925783
Fold: 1  -  55379 13140
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[676]	training's auc: 1	valid_1's auc: 0.944224
Fold: 2  -  58132 10387
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[620]	training's auc: 1	valid_1's auc: 0.960385
Fold: 3  -  58347 10172
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[710]	training's auc: 1	valid_1's auc: 0.958206
Fold: 4  -  59041 9478
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[716]	training's auc: 1	valid_1's auc: 0.941122
Fold: 5  -  59952 8567
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[698]	training's auc: 1	valid_

In [8]:
# test_predictions.to_pickle('../input/mini-model-predictions/predictions_c.pkl') # 0.9437